# Step1 匯入資料

In [34]:
from sklearn.datasets import fetch_mldata
mnist=fetch_mldata('MNIST original')
mnist

{'COL_NAMES': ['label', 'data'],
 'DESCR': 'mldata.org dataset: mnist-original',
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'target': array([0., 0., 0., ..., 9., 9., 9.])}

In [35]:
X, y = mnist["data"], mnist["target"]
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


# Step2 選取70%資料做預測

In [36]:
import numpy as np
shuffle_index = np.random.permutation(70000) #洗牌?
#70000*0.7 = 49000
X_train, X_test, y_train, y_test = X[shuffle_index[:49000]], X[shuffle_index[49000:]], y[shuffle_index[:49000]], y[shuffle_index[49000:]] 

# Step3 平均分配資料

In [37]:
from collections import Counter
count_data= Counter(y_train)
print(count_data)

Counter({1.0: 5527, 7.0: 5077, 3.0: 5014, 9.0: 4915, 2.0: 4860, 6.0: 4811, 0.0: 4809, 8.0: 4800, 4.0: 4742, 5.0: 4445})


# Step4 SVM

In [38]:
import datetime
starttime = datetime.datetime.now()
#long running
from sklearn.svm import SVC
SVC_rbf_clf =  SVC(kernel="rbf", C=2.8, gamma=.0073)

SVC_X_train = X_train/255.0*2 - 1 

SVC_rbf_clf.fit(SVC_X_train, y_train) 
svm_mnist_predictions = SVC_rbf_clf.predict(SVC_X_train)

from sklearn import metrics
accuracy = metrics.accuracy_score(y_train, svm_mnist_predictions)
print (accuracy)

'''
from sklearn.model_selection import  
scores = cross_val_score(SVC_rbf_clf, X_train/255.0*2 - 1, y_train, cv=5, scoring="accuracy")
'''
endtime = datetime.datetime.now()
print ((endtime-starttime))

0.9997755102040816
0:22:45.377007


# Step5 混淆矩陣

In [41]:
import pandas as pd
pd.crosstab(y_train, svm_mnist_predictions,rownames=['label'],colnames=['predict'])

predict,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
label,,,,,,,,,,
0.0,4809,0,0,0,0,0,0,0,0,0
1.0,0,5525,0,0,0,0,0,2,0,0
2.0,0,0,4860,0,0,0,0,0,0,0
3.0,0,0,0,5013,0,0,0,0,0,1
4.0,0,0,0,0,4740,0,0,0,0,2
5.0,0,0,0,0,0,4444,1,0,0,0
6.0,0,0,0,0,0,0,4811,0,0,0
7.0,0,1,0,0,0,0,0,5076,0,0
8.0,0,0,0,0,0,0,0,0,4800,0


# Step6 交叉驗證

In [42]:
import datetime
starttime = datetime.datetime.now()

from sklearn.model_selection import cross_val_score
scores = cross_val_score(SVC_rbf_clf, SVC_X_train, y_train, cv=5, scoring="accuracy")

print(scores)
endtime = datetime.datetime.now()
print ((endtime-starttime))

[0.98102622 0.98449296 0.98377385 0.98418206 0.98346433]
0:49:30.048893


# 存取model

In [45]:
from sklearn.externals import joblib
joblib.dump(SVC_rbf_clf, 'SVC_rbf_clf20180422.pkl') 

['SVC_rbf_clf20180422.pkl']

# 讀取model

In [46]:
from sklearn.externals import joblib
clf = joblib.load('SVC_rbf_clf.pkl') 
print(clf)

SVC(C=2.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0073, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
